In [29]:
jt -r

NameError: name 'jt' is not defined

In [1]:
pip install xgboost

Note: you may need to restart the kernel to use updated packages.


In [2]:
# data analysis and wrangling
import pandas as pd
import numpy as np
import pandas_profiling as pdprof
from dplython import select, DplyFrame, X, arrange, count, sift, head, summarize, group_by, tail, mutate
from math import floor,sqrt
#data modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
import xgboost as xgb

# visualization
import seaborn as sns
import matplotlib.pyplot as plt


C:\Users\Alfonso\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Alfonso\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\Alfonso\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [3]:
training = pd.read_csv("Data/trainingData.csv")
validation = pd.read_csv("Data/validationData.csv")
training.drop_duplicates()
validation.drop_duplicates()

,WAP001,WAP002,WAP003,WAP004,WAP005,WAP006,WAP007,WAP008,WAP009,WAP010,...,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,SPACEID,RELATIVEPOSITION,USERID,PHONEID,TIMESTAMP
0,100,100,100,100,100,100,100,100,100,100,...,100,-7515.916799,4.864890e+06,1,1,0,0,0,0,1380872703
1,100,100,100,100,100,100,100,100,100,100,...,100,-7383.867221,4.864840e+06,4,2,0,0,0,13,1381155054
2,100,100,100,100,100,100,100,100,100,100,...,100,-7374.302080,4.864847e+06,4,2,0,0,0,13,1381155095
3,100,100,100,100,100,100,100,100,100,100,...,100,-7365.824883,4.864843e+06,4,2,0,0,0,13,1381155138
4,100,100,100,100,100,100,100,100,100,100,...,100,-7641.499303,4.864922e+06,2,0,0,0,0,2,1380877774
5,100,100,100,100,100,100,100,100,100,100,...,100,-7338.807210,4.864825e+06,2,2,0,0,0,12,1380874853
6,100,100,100,100,100,100,100,100,100,100,...,100,-7379.351683,4.864849e+06,3,2,0,0,0,20,1380874982
7,100,100,100,100,100,100,100,100,100,100,...,100,-7340.558777,4.864759e+06,3,2,0,0,0,21,1380875085
8,100,100,100,100,100,100,100,100,100,100,...,100,-7357.531253,4.864766e+06,2,2,0,0,0,12,1380875092
9,100,100,100,100,100,100,100,100,100,100,...,100,-7345.085170,4.864831e+06,0,2,0,0,0,20,1380875118


In [4]:
nuniqueTrain = training.apply(pd.Series.nunique)
cols_to_drop_train = nuniqueTrain[nuniqueTrain == 1].index
training.drop(columns = cols_to_drop_train)
validation.drop(columns = cols_to_drop_train)
nuniqueValidation = validation.apply(pd.Series.nunique)
cols_to_drop_validation = nuniqueValidation[nuniqueValidation == 1].index
training = training.drop(columns = cols_to_drop_validation).assign(source = "training")
validation = validation.drop(columns = cols_to_drop_validation).assign(source = "validation")
# it would be better to remove the zero variance in each table and then merge the common columns into
# a new data frame to obsesed with the R way of oding things
fullData = pd.concat([training,validation])
fullData =fullData.drop(columns = ['PHONEID','TIMESTAMP'])
fullData[['FLOOR','BUILDINGID']] = fullData[['FLOOR','BUILDINGID']].astype('category')

In [13]:
# pos_max_wap <- apply(full_data %>% select(starts_with('WAP')), 1,
#                      function(x){which.max(x)})
# list_of_WAPS <- sapply(pos_max_wap, function(x){return(names(full_data)[x])})
# full_data<-full_data %>% mutate(highestWAP=list_of_WAPS)
fullData

,WAP001,WAP003,WAP004,WAP008,WAP009,WAP010,WAP011,WAP012,WAP013,WAP014,...,WAP500,WAP501,WAP502,WAP508,WAP520,LONGITUDE,LATITUDE,FLOOR,BUILDINGID,source
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7541.264300,4.864921e+06,2,1,training
1,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7536.621200,4.864934e+06,2,1,training
2,100,100,100,-97,100,100,100,100,100,100,...,100,100,100,100,100,-7519.152400,4.864950e+06,2,1,training
3,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7524.570400,4.864934e+06,2,1,training
4,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7632.143600,4.864982e+06,0,0,training
5,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7533.896200,4.864939e+06,2,1,training
6,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7519.152400,4.864950e+06,2,1,training
7,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7527.451100,4.864929e+06,2,1,training
8,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7559.497300,4.864888e+06,2,1,training
9,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,-7510.437173,4.864949e+06,2,1,training


In [5]:
dataTrain = fullData[fullData.source == 'training'].drop(columns = 'source')
dataValidation = fullData[fullData.source == 'validation'].drop(columns = 'source')

In [ ]:
X_train1 = dataTrain.filter(regex = '^WAP')
y_train.BUILDINGID = dataTrain.BUILDINGID
X_test1 = dataValidation.filter(regex = '^WAP')
y_test.BUILDINGID = dataValidation.BUILDINGID

In [18]:
RandomForestClassifier_ntree1000 = RandomForestClassifier(n_estimators=1000, random_state=123,
                                                          max_depth=32)

In [17]:
xgb_model_class = xgb.XGBClassifier(objective="multi:softprob", random_state = 123, n_estimators=1000,
                                    early_stopping_rounds=10, n_jobs=3)
xgb_model_reg = xgb.XGBRegressor(objective='reg:linear', random_state = 123, n_estimators=2000,
                                    early_stopping_rounds=10, n_jobs=3)

In [ ]:
RandomForest_BUILDING = RandomForestClassifier_ntree1000.fit(X = X_train1,
                                                             y = y_train.BUILDINGID)

In [ ]:
xgb_BUILDING = xgb_model_class.fit(X_train1, y_train.BUILDINGID, early_stopping_rounds=10,
                                   eval_set=[(X_test1, y_test.BUILDINGID)])

In [16]:
predictions_BUILDING = RandomForest_BUILDING.predict(dataValidation.filter(regex = '^WAP'))
predictions_BUILDING_xgb = xgb_BUILDING.predict(dataValidation.filter(regex = '^WAP'))
print(confusion_matrix(predictions_BUILDING,dataValidation.BUILDINGID))
confusion_matrix(predictions_BUILDING_xgb,dataValidation.BUILDINGID)

[[534   1   0]
 [  2 306   0]
 [  0   0 268]]


array([[534,   0,   0],
       [  2, 307,   0],
       [  0,   0, 268]], dtype=int64)

we try to run the random forest and xgb with and without building so we can see if the results improve by adding the building column we will repeat this process in the next predictions, as the results were better for the xgb model we will add these to the test.

In [ ]:
#we got building going so we add a new column to the predictors
X_train2 = X_train1.assign(BUILDINGID = dataTrain.BUILDINGID)
y_train.LONGITUDE = dataTrain.LONGITUDE
X_test2 = X_train1.assign(BUILDINGID = predictions_BUILDING_xgb)
y_test.LONGITUDE = dataValidation.LONGITUDE

In [ ]:
RandomForest_LONG = RandomForestClassifier_ntree1000.fit(X = X_train1,
                                                           y = y_train.LONGITUDE)

In [ ]:
xgb_BUILDING_LONG = xgb_model_reg.fit(X_train1, y_train.BUILDINGID, early_stopping_rounds=10
                                          eval_set=[(X_test1, y_test.LONGITUDE)])

In [ ]:
RandomForest_LONG_wB = RandomForestClassifier_ntree1000.fit(X = X_train2,
                                                           y = y_train.LONGITUDE)

In [ ]:
xgb_BUILDING_LONG_wB = xgb_model_reg.fit(X_train2, y_train.LONGITUDE, early_stopping_rounds=10
                                          eval_set=[(X_test2, y_test.LONGITUDE)])

In [ ]:
predictions_LONGITUDE_rf = RandomForest_LONG.predict(X_test1)
predictions_LONGITUDE_xgb = xgb_BUILDING_LONG.predict(X_test1)
predictions_LONGITUDE_wB_rf = RandomForest_LONG_wB.predict(X_test2)
predictions_LONGITUDE_wB_xgb = xgb_BUILDING_LONG_wB.predict(X_test2)

print("confusion matrix LONGITUDE_rf")
print(confusion_matrix(predictions_LONGITUDE_rf,dataValidation.LONGITUDE))
print("confusion matrix _LONGITUDE_xgb")
print(confusion_matrix(predictions_LONGITUDE_xgb,dataValidation.LONGITUDE))

print("confusion matrix predictions_LONGITUDE_wB_rf")
print(confusion_matrix(predictions_LONGITUDE_wB_rf,dataValidation.LONGITUDE))
print("confusion matrix predictions_LONGITUDE_wB_xgb")
print(confusion_matrix(predictions_LONGITUDE_wB_xgb,dataValidation.LONGITUDE))

In [ ]:
#we got building going so we add a new column to the predictors
X_train2 = X_train1.assign(BUILDINGID = dataTrain.BUILDINGID)
y_train.LATITUDE = dataTrain.LATITUDE
X_test2 = X_train1.assign(BUILDINGID = predictions_BUILDING_xgb)
y_test.LATITUDE = dataValidation.LATITUDE

In [ ]:
RandomForest_LAT = RandomForestClassifier_ntree1000.fit(X = X_train1,
                                                           y = y_train.LATITUDE)

In [ ]:
xgb_BUILDING_LAT = xgb_model_reg.fit(X_train1, y_train.BUILDINGID, early_stopping_rounds=10
                                          eval_set=[(X_test1, y_test.LATITUDE)])

In [ ]:
RandomForest_LAT_wB = RandomForestClassifier_ntree1000.fit(X = X_train2,
                                                           y = y_train.LATITUDE)

In [ ]:
xgb_BUILDING_LAT_wB = xgb_model_reg.fit(X_train2, y_train.LATITUDE, early_stopping_rounds=10
                                          eval_set=[(X_test2, y_test.LATITUDE)])

In [ ]:
predictions_LATITUDE_rf = RandomForest_LAT.predict(X_test1)
predictions_LATITUDE_xgb = xgb_BUILDING_LAT.predict(X_test1)
predictions_LATITUDE_wB_rf = RandomForest_LAT_wB.predict(X_test2)
predictions_LATITUDE_wB_xgb = xgb_BUILDING_LAT_wB.predict(X_test2)

print("confusion matrix LATITUDE_rf")
print(confusion_matrix(predictions_LATITUDE_rf,dataValidation.LATITUDE))
print("confusion matrix _LATITUDE_xgb")
print(confusion_matrix(predictions_LATITUDE_xgb,dataValidation.LATITUDE))

print("confusion matrix predictions_LATITUDE_wB_rf")
print(confusion_matrix(predictions_LATITUDE_wB_rf,dataValidation.LATITUDE))
print("confusion matrix predictions_LATITUDE_wB_xgb")
print(confusion_matrix(predictions_LATITUDE_wB_xgb,dataValidation.LATITUDE))